# Environment Setup

In [9]:
%pip install -qU google-generativeai
%pip install python-dotenv
%pip install pandas
%pip install ollama
%pip install nltk
# %pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 29.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()

genai.configure(api_key=os.environ['API_KEY'])

model = genai.GenerativeModel('gemini-pro')
response = model.generate_content('Please summarise this document: ...')

print(response.text)

Unfortunately, I do not have access to external documents to summarize for you. Please provide the document's text or a link to it so I can assist you.


In [5]:
import pandas as pd

df = pd.read_csv('chunks/chunk00.csv')

model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")
prompt = """
here's a file that contains my browsing history. I want to preprocess the browsing history so that I can analyze my patterns later. but to do that I need to delete rows that are not relevant like redirect urls or non-informational ones. Filter out the links in this csv files and then return a refined version in the same csv format. remember return in strictly csv format
"""
response = model.generate_content([prompt, df.to_string()])

print(response.text)

URL,Title,Visit Count,Last Visit Time
https://resources.arc.net/hc/en-us/articles/19335089616791-Import-Bookmarks-Logins-History-Extensions-from-Your-Previous-Browser,Import Bookmarks, Logins, History, & Extensions from Your Previous Browser – Arc Help Center,1,2024-04-20 15:38:02
https://www.google.com/search?q=arc+browser+browsing+history+export&sourceid=chrome&ie=UTF-8,arc browser browsing history export - Google Search,2,2024-04-20 15:37:52
https://github.com/ivnvxd/arc-export,ivnvxd/arc-export: Export pinned Arc Browser tabs for importing bookmarks into other browsers,4,2024-04-20 15:37:38
https://www.reddit.com/r/ArcBrowser/comments/178okj5/how_do_i_export_my_arc_data/,How do I export my Arc data? : r/ArcBrowser,2,2024-04-20 15:37:32
https://aistudio.google.com/app/prompts/new_chat?utm_source=gfd&utm_medium=referral&utm_campaign=blog-gemini-15-pro-preview&utm_content=,Untitled prompt | Google AI Studio,3,2024-04-20 15:32:53
https://www.google.com/search?q=export+arc+browsing+hist

# Extracting Browsing History

In [3]:
import sqlite3
import pandas as pd
import os
from shutil import copyfile
import platform

def fetch_chrome_history():
    # Determine the Chrome profile path based on the operating system
    if platform.system() == "Windows":
        # Typically, the path for Chrome history on Windows
        data_path = os.path.expandvars(r"%LOCALAPPDATA%\Google\Chrome\User Data\Default")
    elif platform.system() == "Darwin":
        # Path for macOS
        # data_path = os.path.expanduser("~/Library/Application Support/Google/Chrome/Default")
        data_path = os.path.expanduser("~/Library/Application Support/Arc/User Data/Default")
    else:
        # Path for Linux
        data_path = os.path.expanduser("~/.config/google-chrome/Default")

    # File containing the Chrome history
    history_db = os.path.join(data_path, 'History')

    # Temporary location to copy the History database file
    temp_history = os.path.join(os.getcwd(), 'temp_history')

    # Copying History to current directory to avoid sqlite database is locked error
    copyfile(history_db, temp_history)

    # Connecting to the copied database
    conn = sqlite3.connect(temp_history)
    cursor = conn.cursor()

    # Query to extract urls and visit time in human readable form
    query = """
    SELECT url, title, visit_count, datetime((last_visit_time/1000000)-11644473600, 'unixepoch', 'localtime') AS last_visit_time
    FROM urls ORDER BY last_visit_time DESC
    """

    # Execute the query and fetch all results
    cursor.execute(query)
    data = cursor.fetchall()

    # Convert the data into a pandas DataFrame
    columns = ['URL', 'Title', 'Visit Count', 'Last Visit Time']
    df = pd.DataFrame(data, columns=columns)

    # Save the data to CSV
    # df.to_csv('chrome_history.csv', index=False)
    df.to_csv('arc_history.csv', index=False)

    # Close the connection and remove the temporary file
    cursor.close()
    conn.close()
    os.remove(temp_history)

    print("History has been saved to arc_history.csv")

# Run the function
fetch_chrome_history()


History has been saved to arc_history.csv


# Chunk and refine

## Chunk

In [35]:
import nltk

def count_tokens(sentence):
    tokens = nltk.word_tokenize(sentence)
    return len(tokens)

os.makedirs('chunks', exist_ok=True)

def chunk_csv(file_path, max_tokens=1500000):
    sample = pd.read_csv(file_path, nrows=100)
    # avg_tokens_per_line = sample.apply(lambda x: len(' '.join(x.astype(str)).split()), axis=1).mean()
    # avg_tokens_per_line = count_tokens(sample.to_string())
    avg_tokens_per_line = model.count_tokens(sample.to_string()).total_tokens / 100
    print(f"Average tokens per line: {avg_tokens_per_line}")
    estimated_lines_per_chunk = int(max_tokens / avg_tokens_per_line)

    chunk_number = 0
    for chunk in pd.read_csv(file_path, chunksize=estimated_lines_per_chunk):
        chunk.to_csv(f'chunks/chunk_{chunk_number}.csv', index=False)
        chunk_number += 1
    
    return chunk_number

chunk_num = chunk_csv('arc_history.csv')
chunk_num

Average tokens per line: 197.18


14

## Refine

In [45]:
df = pd.read_csv(f'chunks/chunk_0.csv')
print(df.head().to_csv())

,URL,Title,Visit Count,Last Visit Time
0,https://resources.arc.net/hc/en-us/articles/19335089616791-Import-Bookmarks-Logins-History-Extensions-from-Your-Previous-Browser,"Import Bookmarks, Logins, History, & Extensions from Your Previous Browser – Arc Help Center",1,2024-04-20 15:38:02
1,https://www.google.com/search?q=arc+browser+browsing+history+export&sourceid=chrome&ie=UTF-8,arc browser browsing history export - Google Search,2,2024-04-20 15:37:52
2,https://github.com/ivnvxd/arc-export,ivnvxd/arc-export: Export pinned Arc Browser tabs for importing bookmarks into other browsers,4,2024-04-20 15:37:38
3,https://www.reddit.com/r/ArcBrowser/comments/178okj5/how_do_i_export_my_arc_data/,How do I export my Arc data? : r/ArcBrowser,2,2024-04-20 15:37:32
4,file:///Users/lixiangyi/Downloads/Takeout/Chrome/History.json,,4,2024-04-20 15:33:24



In [47]:
prompt = """
here's a file that contains my browsing history. I want to preprocess the browsing history so that I can analyze my patterns later. but to do that I need to delete rows that are irrelevant like redirect urls or non-informational ones. Filter out the links in this csv files and then return a refined version (the one without unwanted urls) in the same csv format. remember return in strictly csv format
"""
df = pd.read_csv(f'chunks/chunk_0.csv')
print(df.to_csv())
model.count_tokens(df.to_csv())
response = model.generate_content([prompt, df.to_csv()], request_options={"timeout": 6000})
with open(f'refined_chunks/refined_chunk_1.csv', 'w') as f:
    f.write(response.text)

,URL,Title,Visit Count,Last Visit Time
0,https://resources.arc.net/hc/en-us/articles/19335089616791-Import-Bookmarks-Logins-History-Extensions-from-Your-Previous-Browser,"Import Bookmarks, Logins, History, & Extensions from Your Previous Browser – Arc Help Center",1,2024-04-20 15:38:02
1,https://www.google.com/search?q=arc+browser+browsing+history+export&sourceid=chrome&ie=UTF-8,arc browser browsing history export - Google Search,2,2024-04-20 15:37:52
2,https://github.com/ivnvxd/arc-export,ivnvxd/arc-export: Export pinned Arc Browser tabs for importing bookmarks into other browsers,4,2024-04-20 15:37:38
3,https://www.reddit.com/r/ArcBrowser/comments/178okj5/how_do_i_export_my_arc_data/,How do I export my Arc data? : r/ArcBrowser,2,2024-04-20 15:37:32
4,file:///Users/lixiangyi/Downloads/Takeout/Chrome/History.json,,4,2024-04-20 15:33:24
5,https://aistudio.google.com/app/prompts/new_chat?utm_source=gfd&utm_medium=referral&utm_campaign=blog-gemini-15-pro-preview&utm_content=,Untitled prom

In [48]:
prompt = """
here's a file that contains my browsing history. I want to preprocess the browsing history so that I can analyze my patterns later. but to do that I need to delete rows that are irrelevant like redirect urls or non-informational ones. Filter out the links in this csv files and then return a refined version in the same csv format. remember return in strictly csv format
"""

for i in range(chunk_num):
    df = pd.read_csv(f'chunks/chunk_{i}.csv')
    response = model.generate_content([prompt, df.to_csv()], request_options={"timeout": 6000})
    # put the refined data in new csv file in refine_chunks folder
    with open(f'refined_chunks/refined_chunk_{i}.csv', 'w') as f:
        f.write(response.text)

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import os
import ollama

# Create a directory for the refined chunks
os.makedirs('refined_chunks', exist_ok=True)

def process_with_ollama(chunk_path):
    df = pd.read_csv(chunk_path)
    messages = [{'role': 'user', 'content': str(row['URL']) + " " + str(row['Title'])} for index, row in df.iterrows()]
    
    # Assuming 'llama2' is the correct model configured properly with Ollama
    response = ollama.chat(model='gemma:7b', messages=messages)
    filtered_content = [msg for msg in response if 'important' in msg['content']]  # Example condition
    refined_df = pd.DataFrame(filtered_content)
    return refined_df

def refine_chunks(num_chunks):
    for i in range(num_chunks):
        refined_df = process_with_ollama(f'chunks/chunk_{i}.csv')
        refined_df.to_csv(f'refined_chunks/refined_chunk_{i}.csv', index=False)

# Path to your large CSV file
path_to_csv = 'arc_history.csv'
num_chunks = chunk_csv(path_to_csv)
refine_chunks(num_chunks)


```tool_code
print('Hello world!')
```


# Page summarization with local models

# Chunk and search

# Hybrid -- cascading